In [1]:
import scipy.io
import tensorflow as tf
import numpy as np
from tensorflow import  keras
from keras.callbacks import TensorBoard
from sklearn.preprocessing import MinMaxScaler,StandardScaler,scale
from skimage.measure import block_reduce
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout

In [2]:
def load_data():
    als = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/ALS.mat',)["data"].transpose()  #(262134, 321)
    normal = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/normal.mat')['data'].transpose() # (262134, 270) 
    myopathie = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/myopathie.mat')['data'].transpose() #(262134, 315)
    return als,normal,myopathie

In [3]:

def scale_data_minmax(data):
#type of scaler that scales the minimum and maximum values to be 0 and 1 respectively. 
    scaler = MinMaxScaler()
#fit_transform() then it will calculate the mean(μ) and standard deviation(σ) of the feature F at a time it will transform the data points of the feature F.    
    normalized_data =scaler.fit_transform(data)


    return normalized_data

In [4]:
def scale_data_standard(data):
    scaled_data = scale(data,axis=1)
    return scaled_data

In [5]:
def gen_tensors_list(data,dim):
    dataset  = list()
    for i in range (len(data)):
        arg = tf.convert_to_tensor(data[i], dtype=tf.float32)
        arg = tf.reshape(arg,dim)
        dataset.append(arg)

    return dataset

In [6]:
def windowing_fun(data,window_size=1000,overlap=100):
    windowed_data = [data[j][i : i + window_size] for j in range(0,len(data)) for i in range(0, len(data[j]), window_size-overlap)]
    final_data = [windowed_data[i] for i in range(len(windowed_data)) if len(windowed_data[i]) == window_size ]
    return final_data

In [7]:
def load_file_names():
    als = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/ALS.mat',)["files"]  #(262134, 321)
    normal = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/normal.mat')['files'] # (262134, 270) 
    myopathie = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/myopathie.mat')['files'] #(262134, 315)
    return als,normal,myopathie

In [33]:
def generate_dataframe(data,files):
    df = pd.DataFrame([], columns=['type_signal',"num_personne","muscle","num_enregistrement","signal"])
    windowed_data = [{"signal": data[j],"type_signal":files[0][j][0][5],"num_personne": files[0][j][0][6:8],"muscle": files[0][j][0][8:10],"num_enregistrement":files[0][j][0][10:12] } for j in range(0,len(data)) ]
    final_data = [windowed_data[i] for i in range(len(windowed_data))]
    df =df.append(final_data, ignore_index=True)
    #df =pd.concat([df, final_data])
    #df = pd.concat([df, pd.DataFrame.from_records(final_data)], ignore_index=True)
    return df

In [34]:
als, normal , myopathie = load_data()

In [35]:
alsfiles,normalfiles,myopathiefiles = load_file_names()

In [36]:
df = generate_dataframe(np.vstack((normal,als,myopathie)),np.concatenate((normalfiles,alsfiles,myopathiefiles),axis=1))

C:\Users\lenovo\AppData\Local\Temp\ipykernel_5984\1494603022.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df =df.append(final_data, ignore_index=True)


In [37]:
def f(row):
    return {"C":0,"A":1,"M":2}[row["type_signal"]]

df["num_class"] =df.apply(f, axis=1) 

In [38]:
df = df.query("muscle=='BB'")

In [39]:
X = df['signal'].tolist()
Y = df['num_class'].tolist()

In [40]:
X=X[170:]
Y=Y[170:]

In [41]:
del df

In [42]:
random_noise_1 = np.random.normal(0,1,len(X[0]))
random_noise_2 = np.random.normal(0,1,len(X[0]))
random_noise_3 = np.random.normal(0,1,len(X[0]))
random_noise_4 = np.random.normal(0,1,len(X[0]))
np.shape(random_noise_1)

(262134,)

In [43]:
data = [*X]
#data, np.shape(data)

In [44]:
data  += list(map(lambda x : x + random_noise_1, X))
data  += list(map(lambda x : x + random_noise_2, X))
data  += list(map(lambda x : x + random_noise_3, X))
#data  += list(map(lambda x : x + random_noise_4, X))
print(np.asarray(data).shape)
#print(data)

(1220, 262134)


In [45]:
del  als,normal,myopathie,alsfiles,normalfiles,myopathiefiles 

In [46]:
data = scale_data_standard(data)

In [47]:
last_index_normal = len(Y) - 1 - Y[::-1].index(0)

In [48]:
last_index_als = len(Y) - 1 - Y[::-1].index(1)

In [49]:
last_index_myopathie = len(Y) - 1 - Y[::-1].index(2)

In [50]:
windowed_normal = len(windowing_fun(X[:last_index_normal+1],10000))
windowed_als = len(windowing_fun(X[last_index_normal+1:last_index_als+1],10000))
windowed_myopathie = len(windowing_fun(X[last_index_als+1:],10000))

In [51]:
Y = [0]*(windowed_normal)+[1]*(windowed_als)+[2]*(windowed_myopathie)

In [52]:
Y  = Y*4

In [53]:
X = windowing_fun(data,10000)

In [54]:
del data

In [55]:
X = scale_data_standard(X)

In [56]:
tensors_list = gen_tensors_list(X,(1,10000))

In [57]:
X_train, X_test, y_train, y_test = train_test_split( tensors_list, Y, test_size=0.04,random_state=20)
np.shape(X_train), np.shape(X_test)

((30451, 1, 10000), (1269, 1, 10000))

In [58]:
model = Sequential([
    #Conv2D(#of filters, filters size, activation function)
    #in keras for the first layer we always need to mention the input shape
    Conv1D(64, 3, activation = 'relu', padding ="same", input_shape = (1,10000)), 
    
    #MaxPooling2D(filter size stride and pad can also be added)
    MaxPooling1D(2, strides = 2, padding ="same"),
    
    #LAYER2
    Conv1D(32, 3, activation = 'relu', padding ="same"),
    MaxPooling1D(2, strides = 2, padding ="same"),
    
    #FLATTEN
    Flatten(),
    
    #FC Layers:
    
    #LAYER3
    #Dense : fully connected  Dense(#neurons, activation function )
    Dense(64, activation = 'relu'),
    Dropout(0.6),
    
    #LAYER4
    Dense(3, activation = 'softmax'),
    
    
])


In [59]:
logdir = "drive/MyDrive/PFE_TENSORBOARD_DATA/augmentation0/CNN1D/logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks = [ tf.keras.callbacks.TensorBoard(log_dir=logdir),tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy',patience =60 ,min_delta=1e-3)]


In [60]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [61]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_text = np.asarray(X_test)
y_test = np.asarray(y_test)

In [62]:
model.fit(X_train, y_train, epochs =50, batch_size = 128)

Epoch 1/50
238/238 [==============================] - 35s 129ms/step - loss: 1.0502 - accuracy: 0.4479
Epoch 2/50
238/238 [==============================] - 29s 121ms/step - loss: 0.7240 - accuracy: 0.6681
Epoch 3/50
238/238 [==============================] - 29s 123ms/step - loss: 0.4572 - accuracy: 0.8081
Epoch 4/50
238/238 [==============================] - 29s 124ms/step - loss: 0.3180 - accuracy: 0.8739
Epoch 5/50
238/238 [==============================] - 30s 127ms/step - loss: 0.2233 - accuracy: 0.9168
Epoch 6/50
238/238 [==============================] - 29s 123ms/step - loss: 0.1889 - accuracy: 0.9296
Epoch 7/50
238/238 [==============================] - 30s 124ms/step - loss: 0.1371 - accuracy: 0.9482
Epoch 8/50
238/238 [==============================] - 32s 136ms/step - loss: 0.1356 - accuracy: 0.9516
Epoch 9/50
238/238 [==============================] - 31s 130ms/step - loss: 0.1140 - accuracy: 0.9587
Epoch 10/50
238/238 [==============================] - 32s 134ms/step - l

In [63]:
score = model.evaluate(np.asarray(X_test),np.asarray (y_test))

40/40 [==============================] - 1s 6ms/step - loss: 0.1380 - accuracy: 0.9803


In [198]:
%reload_ext tensorboard

In [199]:
%tensorboard --logdir drive/MyDrive/PFE_TENSORBOARD_DATA/augmentation0/CNN1D/logs/scalars/

Reusing TensorBoard on port 6006 (pid 12412), started 4:22:36 ago. (Use '!kill 12412' to kill it.)

In [ ]:
logdir = "drive/MyDrive/PFE_TENSORBOARD_DATA/augmentation0/CNN1D/logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks = [ tf.keras.callbacks.TensorBoard(log_dir=logdir),tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy',patience =60 ,min_delta=1e-3)]

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
history = model.fit(
    
    x = np.asarray(X_train)  ,
    y= np.asarray(y_train) ,
    validation_split=0.1,
    epochs=50,
    batch_size=64,
    callbacks=callbacks

    )

In [ ]:
score = model.evaluate(np.asarray(X_test),np.asarray (y_test) ) 

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir drive/MyDrive/PFE_TENSORBOARD_DATA/augmentation0/CNN1D/logs/scalars/

In [ ]:
model.save("/content/drive/MyDrive/PFE_RESULTS_DATA/augmentation0/scaling/CNN1D/CNN1D.h5")

In [ ]:
import json

with open('/content/drive/MyDrive/PFE_RESULTS_DATA/augmentation0/scaling/CNN1D/CNN1D_score.json', 'w+') as file:

    json.dump(score, file)

In [ ]:
model= tf.keras.models.load_model("/content/drive/MyDrive/PFE_RESULTS_DATA/augmentation0/scaling/CNN1D/CNN1D.h5")

In [ ]:
tf.keras.utils.plot_model(
    model,
    show_shapes=True
    )

In [ ]:
y_pred = model.predict(np.asarray(X_test))

In [ ]:
y_predicted_labels = [{0:"Normal",1:"Neuropathie",2:"Myopathie"}[i] for i in np.argmax(y_pred,axis=1)]

In [ ]:
y_true_labels = [{0:"Normal",1:"Neuropathie",2:"Myopathie"}[i] for i in y_test]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

con_mat = tf.math.confusion_matrix(labels=y_test, predictions=np.argmax(y_pred,axis=1)).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = ["Normal","Neuropathie"], 
                     columns = ["Normal","Neuropathie"])

figure = plt.figure(figsize=(5,5))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()
